In [ ]:
#| default_exp pymor.example_problems

# pymor.example_problems

> Extended functionality for pyMOR

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import test_eq, test_close

In [ ]:
# #| export
# from fastcore.basics import patch, first

# from typing import Tuple, List
# from numbers import Integral
# from IPython.display import display, HTML

# import pandas as pd
# from pandas import Index

# from numpy import array, ndarray
# from einops import rearrange
# import plotly
# import plotly.express as px

# from pymor.core.base import ImmutableObject
# from pymor.basic import NumpyVectorSpace
# from pymor.vectorarrays.block import BlockVectorArray, BlockVectorSpace, BlockVectorArrayImpl

## LTI example problem

In [ ]:
#| export
import numpy as np
import scipy.sparse as sps
from pymor.models.iosys import LTIModel

#| hide
### lti_matrices

In [ ]:
#| export
def lti_matrices():
    k = 50
    n = 2 * k + 1
    
    E = sps.eye(n, format='lil')
    E[0, 0] = E[-1, -1] = 0.5
    E = E.tocsc()
    
    d0 = n * [-2 * (n - 1)**2]
    d1 = (n - 1) * [(n - 1)**2]
    A = sps.diags([d1, d0, d1], [-1, 0, 1], format='lil')
    A[0, 0] = A[-1, -1] = -n * (n - 1)
    A = A.tocsc()
    
    B = np.zeros((n, 2))
    B[:, 0] = 1
    B[0, 0] = B[-1, 0] = 0.5
    B[0, 1] = n - 1
    
    C = np.zeros((3, n))
    C[0, :n//3] = C[1, n//3:2*n//3] = C[2, 2*n//3:] = 1
    C /= C.sum(axis=1)[:, np.newaxis]
    return A, B, C, E

In [ ]:
A, B, C, E = lti_matrices()

In [ ]:
fom = LTIModel.from_matrices(A, B, C, E=E)

In [ ]:
fom

LTIModel(
    NumpyMatrixOperator(<101x101 sparse, 301 nnz>, source_id='STATE', range_id='STATE'),
    NumpyMatrixOperator(<101x2 dense>, range_id='STATE'),
    NumpyMatrixOperator(<3x101 dense>, source_id='STATE'),
    D=ZeroOperator(NumpyVectorSpace(3), NumpyVectorSpace(2)),
    E=NumpyMatrixOperator(<101x101 sparse, 101 nnz>, source_id='STATE', range_id='STATE'),
    presets={})

In [ ]:
fom.B.source

NumpyVectorSpace(2)

The two inputs are $u_1$ (uniform heating), $u_2$ (heating the left end)

In [ ]:
fom.C.range

NumpyVectorSpace(3)

The three outputs are $y_1$ (left side temperature), $y_2$ (central temperature), $y_3$ (right side temperature)

In [ ]:
fom.A.source

NumpyVectorSpace(101, id='STATE')

There are 101 grid positions $\xi = 0, ..., 1$

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()